In [1]:
%pylab nbagg
from tvb.simulator.lab import *
import pandas as pd
import h5py

from tvb.simulator.lab import *
import h5py
import os
import matplotlib.pyplot as plt

import scipy 
import scipy.fftpack
import seaborn as sns

import scipy 
import scipy.fftpack
from scipy.fft import fft, ifft
from scipy import signal
from scipy.integrate import simps
from fooof import FOOOF

import warnings
warnings.filterwarnings("ignore")

Populating the interactive namespace from numpy and matplotlib


In [2]:
#depressed brain 
Qvals= [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0]
Qvals = np.array(Qvals)
oscilator = models.WilsonCowan(Q = Qvals)

In [3]:
oscilator = models.WilsonCowan(c_ee = np.repeat(14.05, 76), c_ei = np.repeat(12.44, 76), c_ie = np.repeat(16.76, 76),
                              c_ii = np.repeat(2.0, 76), tau_e = np.repeat(16.07, 76), tau_i = np.repeat(33.71, 76),
                              a_e = np.repeat(1.3, 76), b_e = np.repeat(4.0, 76), c_e = np.repeat(1.0, 76),
                              a_i = np.repeat(1.95, 76), b_i = np.repeat(4.76, 76), c_i = np.repeat(1.0, 76),
                              r_e = np.repeat(1.0, 76), r_i = np.repeat(1.0, 76), k_e = np.repeat(1.0, 76),
                              k_i = np.repeat(1.0, 76), P = np.repeat(2.22, 76), Q = Qvals, theta_e = np.repeat(0.0, 76),
                              theta_i = np.repeat(0.0, 76), alpha_e = np.repeat(1.0, 76), alpha_i = np.repeat(1.0, 76))

In [4]:
eqn_t = equations.PulseTrain()
eqn_t.parameters['onset'] = 0.0
eqn_t.parameters['T'] = 1000.0
eqn_t.parameters['tau'] = 1.0
eqn_t.parameters["amp"] = 0.035

In [6]:
# configure stimulus spatial pattern: LF
weighting = numpy.zeros((76, ))
weighting[[18]] = 5   #LF

In [7]:
white_matter = connectivity.Connectivity.from_file()
white_matter.speed = numpy.array([3.0])

WARNING  File 'hemispheres' not found in ZIP.


In [8]:
white_matter_coupling = coupling.Linear(a=numpy.array([0.0039]))

In [9]:
stimulus = patterns.StimuliRegion(
    temporal=eqn_t,
    connectivity=white_matter,
    weight=weighting)

In [10]:
#Initialise some Monitors with period in physical time
mon_raw = monitors.Raw(period = 0.9765625)
mon_tavg = monitors.TemporalAverage(period=1)

#Bundle them
what_to_watch = (mon_raw, mon_tavg)

In [11]:
#important variables of dataset
sample_period = 0.0009765625 #seconds         
sf = 1/sample_period #sampling frequency 
sample_rate = 1024

#sns.set(font_scale=1.2)
dt = 0.01

In [12]:
os.chdir("C:/Users/celie/OneDrive - ugentbe/NOG NODIG/2de master/Masterproef/Wilson Cowan/DATA")
#import connectivity file to know which region corresponds to which index in the matrices
filename = "Connectivity.h5"

with h5py.File(filename, "r") as f:
    # List all groups
    print("Keys: %s" % f.keys())
    a_group_key = list(f.keys())[0]

    # Get the data
    regions_list = list(f['region_labels'])

regions = np.array(regions_list)


Keys: <KeysViewHDF5 ['areas', 'centres', 'cortical', 'hemispheres', 'orientations', 'region_labels', 'tract_lengths', 'weights']>


In [13]:
def check_nan(data):
    array_sum = np.sum(data)
    array_has_nan = np.isnan(array_sum)
    if array_has_nan == True:
        NAN = True
    else:
        NAN = False
    return NAN

In [14]:
def powercalc(data, low, high, sf, win):
    freqs, psd = signal.welch(data, sf, nperseg=win)
    freq_res = freqs[1] - freqs[0]
    total_power = simps(psd, dx=freq_res)
    idx= np.logical_and(freqs >= low, freqs <= high)
    power = simps(psd[idx], dx=freq_res)
    return power

In [16]:
#loop  start om 19u10
mydir = "C:/Users/celie/OneDrive - ugentbe/NOG NODIG/2de master/Masterproef/Datasets/All_results/"

c1 = np.repeat("leavespaceforaword", 38200) #should have 38178 rows, but taking 38200 just to be save
alfa = np.column_stack([c1, c1,c1,c1,c1,c1,c1,c1,c1,c1])

c2 = np.repeat("leavespaceforaword", 3000) #should have 2828 rows, but taking 2840 just to be save 
peak_ar = np.column_stack([c2, c2,c2,c2,c2,c2,c2,c2])


c3 = np.repeat("", 1)
c4 = np.repeat("",1)

ROI_base = numpy.zeros(shape=(1,2))
ROI_ar = np.column_stack([c3, c4, ROI_base])
ROI_tot= np.column_stack([c3, c4, ROI_base])


varkeep_8 = {}
varkeep_9 = {}
varkeep_10 = {}
varkeep_11= {}
varkeep_alfa= {}


c1 = np.repeat("", 1)
c2 = np.repeat("",1)

ROI_base = numpy.zeros(shape=(1,2))
ROI_ar = np.column_stack([c1, c2, ROI_base])
ROI_tot= np.column_stack([c1, c2, ROI_base])

power_arr = numpy.zeros(shape = (1,5))
power_arr_right =numpy.zeros(shape = (1,5))
power_arr_left = numpy.zeros(shape = (1,5))
tot_power_arr = numpy.zeros(shape = (1,5))
tot_power_arr_right =numpy.zeros(shape = (1,5))
tot_power_arr_left = numpy.zeros(shape = (1,5))

varkeep = {}
varhigh = {}
varall = {}
TSkeep = {}
leftDLPFC_keep = {}
rightDLPFC_keep = {}

indices = [7,  13, 15, 16, 17,18, 19, 20, 21, 30, 31, 33,35, 36, 45, 51, 53, 54, 55, 56, 57, 58, 59, 68, 69, 71 , 73,74]
counter = 0
loop = 1
peakcount = 0

NS_arr = np.arange(0,26, 0.25)

win = 4 * sf
sf = 1024

freq_range = [8,12] 

nsubs = 101
for i in range(nsubs):
    if i == 0:
        pass 
    else:
        value = 10**-6
        print(loop)
        
        NS = NS_arr[i] 
        
        #create simulator 
        sim = simulator.Simulator(model = oscilator, connectivity = white_matter,
                          coupling = white_matter_coupling, 
                          integrator = integrators.EulerStochastic(dt=0.01220703125, noise=noise.Additive(noise_seed = int(NS), nsig=numpy.array([value]))),
                        monitors =  what_to_watch, stimulus = stimulus)

        sim.configure()

        print("check1")
        
        #Perform the simulation
        tavg_data = []
        tavg_time = []

        for raw, tavg in sim(simulation_length=2500):
    
            if not tavg is None:
                tavg_time.append(tavg[0])
                tavg_data.append(tavg[1])
    
        #Make the lists numpy.arrays for easier use.
        RAW = numpy.array(tavg_data)
        RAW_real = RAW[:, 0, :, 0]
        raw_time = tavg_time
        
        #delete the first 500ms
        list_deleterows = [*range(0,500,1)]
        RAW_real = np.delete(RAW_real, list_deleterows, 0)
        raw_time = np.delete(raw_time, list_deleterows, 0)
        
        print("check2")
        
        #save average time series (across regions)
        AV = RAW_real
        AV_ar = AV.mean(axis = 1) 
        TSkeep["data" + str(i)] = AV_ar
        
        #save time series of left DLPFC (for alpha asymmetry later)
        leftDLPFC = RAW_real[:,56]
        leftDLPFC_keep["data" + str(i)] = leftDLPFC

        #save time series of right DLPFC (for alpha asymmetry later)
        rightDLPFC = RAW_real[:,18]
        rightDLPFC_keep["data"+str(i)] = rightDLPFC
        
        #store PSDs
        #psd for all brain 
        for x in range(5):            
            low, high = 8+x, 9+x
            if x == 4:
                low, high = 8, 12
            
            power_arr[0][x] = powercalc(AV_ar, low, high, sf, win)
            power_arr_left[0][x] = powercalc(leftDLPFC, low, high, sf, win)
            power_arr_right[0][x] = powercalc(rightDLPFC, low, high, sf, win)
            
        if loop == 1:
            tot_power_arr = numpy.copy(power_arr)
            tot_power_arr_left = numpy.copy(power_arr_left)
            tot_power_arr_right = numpy.copy(power_arr_right)
        else:
            tuplePower = (tot_power_arr, power_arr)
            tot_power_arr = np.vstack(tuplePower)
                
            tuplePower_left = (tot_power_arr_left, power_arr_left)
            tot_power_arr_left = np.vstack(tuplePower_left)
                
            tuplePower_right = (tot_power_arr_right, power_arr_right)
            tot_power_arr_right = np.vstack(tuplePower_right)
        
        counter_coh = 0
        for ROI1 in indices:
            #save power spectrum of left DLPFC
            ROIdata = RAW_real[:,ROI1]
        
            freqs_roi, psd_roi = signal.welch(ROIdata, sf, nperseg = win)
        
            fm = FOOOF(peak_width_limits=[1, 8])
            fm.fit(freqs_roi, psd_roi, freq_range)
            if not fm.has_model:
                print("model fitting failed")
                Peak = "Failed"
                Power = "Failed"
                Peak_width = "Failed"
                Amplitude = "Failed"
            else:
                peaks_roi = fm.get_params('peak_params')
        
                if check_nan(peaks_roi) == True:
                    Peak = "none"
                    Power = "none"
                    Peak_width = "none"
                    Amplitude = "none"
                else:
                    Peak = peaks_roi[0][0]
                    Power = peaks_roi[0][1]
                    Peak_width = peaks_roi[0][2]
                    Amplitude = Power**2
                
            peak_ar[peakcount][0] = i
            peak_ar[peakcount][1] = "depressed"
            peak_ar[peakcount][2] = "LF"
            peak_ar[peakcount][3] = str(regions[ROI1])
            peak_ar[peakcount][4] = Peak
            peak_ar[peakcount][5] = Power
            peak_ar[peakcount][6] = Amplitude
            peak_ar[peakcount][7] = Peak_width
            
            
            peakcount = peakcount + 1
            
            for ROI2 in indices:
                if ROI1 == ROI2:
                    pass
                elif ROI1 > ROI2:
                    pass
                else:
                    x = RAW_real[:,ROI1]
                    y = RAW_real[:,ROI2]
                    
                    cxy, f = cohere(x, y, 256, 1. / dt)
                    cxy_alfa, f_alfa = cxy[21:32], f[21:32] #8-12Hz
                    cxy_8, f_8 = cxy[21:24], f[21:24] #band 8-9Hz
                    cxy_9, f_9 = cxy[24:26], f[24:26] # 9-10Hz
                    cxy_10, f_10 = cxy[26:29], f[26:29] #10-11Hz
                    cxy_11, f11 = cxy[29:32], f[29:32] #11-12Hz
                    cxy_alfa, f_alfa = cxy[21:32], f[21:32] #8-12Hz
                    
                    alfa[counter][0] = i #subject id
                    alfa[counter][1] = "depressed"
                    alfa[counter][2] = "LF"
                    alfa[counter][3] = str(regions[ROI1])
                    alfa[counter][4] = str(regions[ROI2])
                    alfa[counter][5] = mean(cxy_alfa)
                    alfa[counter][6] = mean(cxy_8)
                    alfa[counter][7] = mean(cxy_9)
                    alfa[counter][8] = mean(cxy_10)
                    alfa[counter][9] = mean(cxy_11)
                    
                                        #make matrix of coherence measures
                    coh_8 = cxy_8
                    coh_9 = cxy_9
                    coh_10 = cxy_10
                    coh_11= cxy_11
                    coh_alfa= cxy_alfa
                        
                    if counter_coh == 0:
                        coh_tot_8 = coh_8
                        coh_tot_9 = coh_9
                        coh_tot_10 = coh_10
                        coh_tot_11= coh_11
                        coh_tot_alfa = coh_alfa
                        
                        if loop == 1:
                            ROI_tot[0][0] = str(regions[ROI1])
                            ROI_tot[0][1] = str(regions[ROI2])
                            ROI_tot[0][2] = ROI1
                            ROI_tot[0][3] = ROI2
            

                    else:
                        tuplex = (coh_tot_8, coh_8)
                        coh_tot_8 = np.vstack(tuplex)
                        
                        tuplex = (coh_tot_9, coh_9)
                        coh_tot_9 = np.vstack(tuplex)
                        
                        tuplex = (coh_tot_10, coh_10)
                        coh_tot_10 = np.vstack(tuplex)
                        
                        tuplex = (coh_tot_11, coh_11)
                        coh_tot_11 = np.vstack(tuplex)
                        
                        
                        tuplex = (coh_tot_alfa, coh_alfa)
                        coh_tot_alfa = np.vstack(tuplex)
                        
                        if loop == 1:
                            ROI_ar[0][0] = str(regions[ROI1])
                            ROI_ar[0][1] = str(regions[ROI2])
                            ROI_ar[0][2] = ROI1
                            ROI_ar[0][3] = ROI2
                            tupleROI = (ROI_tot, ROI_ar)
                            ROI_tot = np.vstack(tupleROI)
                            
                    counter_coh = counter_coh + 1
                    
                    counter = counter + 1
    
            
        
        #load in depressed brains 
        varkeep_8["data" + str(i)] = coh_tot_8
        varkeep_9["data" + str(i)] = coh_tot_9
        varkeep_10["data" + str(i)] = coh_tot_10
        varkeep_11["data" + str(i)] = coh_tot_11
        varkeep_alfa["data" + str(i)] = coh_tot_alfa
        
        loop = loop + 1
        
        
        
print('prepare to stack')

file_name = mydir + "/LF_coherence.csv"
outcome = pd.DataFrame.from_records(alfa)
outcome.columns = ["Subject", "Group", "Treatment", "ROI1", "ROI2", "whole band", "8-9Hz","9-10Hz","10-11Hz","11-12Hz"]
outcome.to_csv(path_or_buf = file_name, index = False)

file_name = mydir + "/LF_peak.csv"
outcome = pd.DataFrame.from_records(peak_ar)
outcome.columns = ["Subject", "Group", "Treatment", "ROI1", "frequency", "power", "amplitude", "Bandwith peak"]
outcome.to_csv(path_or_buf = file_name, index = False)

#coherence 8_9Hz
namelist = []
for i in range(nsubs):
    if i == 0:
        pass
    else:
        name = varkeep_8["data" + str(i)] 
        namelist.append(name)

nametup = tuple(namelist)
array_8 = np.hstack(nametup)
        
file_name = mydir + "/LF_coh_total_hstacked_8" + str(i) + ".h5"
h5f = h5py.File(file_name, 'w')
h5f.create_dataset('data', data=array_8)
h5f.close()

#coherence 9_10Hz
namelist = []
for i in range(nsubs):
    if i == 0:
        pass
    else:
        name = varkeep_9["data" + str(i)] 
        namelist.append(name)

nametup = tuple(namelist)
array_9 = np.hstack(nametup)
        
file_name = mydir + "/LF_coh_total_hstacked_9" + str(i) + ".h5"
h5f = h5py.File(file_name, 'w')
h5f.create_dataset('data', data=array_9)
h5f.close()

#coherence 10-11Hz
namelist = []
for i in range(nsubs):
    if i == 0:
        pass
    else:
        name = varkeep_10["data" + str(i)] 
        namelist.append(name)

nametup = tuple(namelist)
array_10 = np.hstack(nametup)
        
file_name = mydir + "/LF_coh_total_hstacked_10" + str(i) + ".h5"
h5f = h5py.File(file_name, 'w')
h5f.create_dataset('data', data=array_10)
h5f.close()

#coherence 11-12Hz
namelist = []
for i in range(nsubs):
    if i == 0:
        pass
    else:
        name = varkeep_11["data" + str(i)] 
        namelist.append(name)

nametup = tuple(namelist)
array_11 = np.hstack(nametup)
        
file_name = mydir + "/LF_coh_total_hstacked_11" + str(i) + ".h5"
h5f = h5py.File(file_name, 'w')
h5f.create_dataset('data', data=array_11)
h5f.close()

#coherence 8-12
namelist = []
for i in range(nsubs):
    if i == 0:
        pass
    else:
        name = varkeep_alfa["data" + str(i)] 
        namelist.append(name)

nametup = tuple(namelist)
array_alfa = np.hstack(nametup)
        
file_name = mydir + "/LF_coh_total_hstacked_alfa" + str(i) + ".h5"
h5f = h5py.File(file_name, 'w')
h5f.create_dataset('data', data=array_alfa)
h5f.close()


#store order of pairs 

file_name = mydir + "/LF_ROI_order.csv"
outcome = pd.DataFrame.from_records(ROI_tot)
outcome.columns = ["ROI1", "ROI2", "ROI1_nr", "ROI2_nr"]
outcome.to_csv(path_or_buf = file_name, index = False)


        
file_name = mydir + "/LF_Time.h5"
h5f = h5py.File(file_name, 'w')
h5f.create_dataset('time', data=raw_time)
h5f.close()


f_array  = f[21:32]
file_name = mydir + "/LF_Frequencies.h5"
h5f = h5py.File(file_name, 'w')
h5f.create_dataset('data', data=f_array)
h5f.close()

#compute complete array containing all time series of all 100simulations   
namelist4 = []
for i in range(nsubs): #range(101):
    if i == 0:
        pass
    else:
        name4 = TSkeep["data" + str(i)]
        namelist4.append(name4)

nametup4 = tuple(namelist4)
array_TS = np.column_stack(nametup4)

print(shape(array_TS))

#make dataset containing all 100 right DLPFC time series + average across them
namelist5 = []
for i in range(nsubs): #range(101):
    if i == 0:
        pass
    else:
        name5 = rightDLPFC_keep["data"+str(i)] 
        namelist5.append(name5)

nametup5 = tuple(namelist5)
array_rDLPFC = np.column_stack(nametup5)


#make dataset containing all 100 left DLPFC time series 
namelist6 = []
for i in range(nsubs): #range(101):
    if i == 0:
        pass
    else:
        name6 = leftDLPFC_keep["data"+str(i)] 
        namelist6.append(name6)

nametup6 = tuple(namelist6)
array_lDLPFC = np.column_stack(nametup6)

file_name = mydir + "/LF_power_alpha_overall.h5"
h5f = h5py.File(file_name, 'w')
h5f.create_dataset('data', data=tot_power_arr)
h5f.close()

file_name = mydir + "/LF_power_alpha_rDLPFC.h5"
h5f = h5py.File(file_name, 'w')
h5f.create_dataset('data', data=tot_power_arr_right)
h5f.close()

file_name = mydir + "/LF_power_alpha_lDLPFC.h5"
h5f = h5py.File(file_name, 'w')
h5f.create_dataset('data', data=tot_power_arr_left)
h5f.close()

file_name = mydir + "/LF_rightDLPFC.h5"
h5f = h5py.File(file_name, 'w')
h5f.create_dataset('data', data=array_rDLPFC)
h5f.close()

file_name = mydir + "/LF_leftDLPFC.h5"
h5f = h5py.File(file_name, 'w')
h5f.create_dataset('data', data=array_lDLPFC)
h5f.close()
        
file_name = mydir + "/LF_Time.h5"
h5f = h5py.File(file_name, 'w')
h5f.create_dataset('time', data=raw_time)
h5f.close()

file_name = mydir + "/LF_TimeSeries.h5"
h5f = h5py.File(file_name, 'w')
h5f.create_dataset('time', data=array_TS )
h5f.close()


1
check1
check2
2
check1
check2
3
check1
check2
4
check1
check2
5
check1
check2
6
check1
check2
7
check1
check2
8
check1
check2
9
check1
check2
10
check1
check2
11
check1
check2
12
check1
check2
13
check1
check2
14
check1
check2
15
check1
check2
16
check1
check2
17
check1
check2
18
check1
check2
19
check1
check2
20
check1
check2
21
check1
check2
22
check1
check2
23
check1
check2
24
check1
check2
25
check1
check2
26
check1
check2
27
check1
check2
28
check1
check2
29
check1
check2
30
check1
check2
31
check1
check2
32
check1
check2
33
check1
check2
34
check1
check2
35
check1
check2
36
check1
check2
37
check1
check2
38
check1
check2
39
check1
check2
40
check1
check2
41
check1
check2
42
check1
check2
43
check1
check2
44
check1
check2
45
check1
check2
46
check1
check2
47
check1
check2
48
check1
check2
49
check1
check2
50
check1
check2
51
check1
check2
52
check1
check2
53
check1
check2
54
check1
check2
55
check1
check2
56
check1
check2
57
check1
check2
58
check1
check2
59
check1
check2
60
che

In [ ]:
#started 19u06